In [16]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'cs231n/assignment1/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/cs231n/assignment1/cs231n/datasets
/content/drive/My Drive/cs231n/assignment1


# Image features exercise
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

We have seen that we can achieve reasonable performance on an image classification task by training a linear classifier on the pixels of the input image. In this exercise we will show that we can improve our classification performance by training linear classifiers not on raw pixels but on features that are computed from the raw pixels.

All of your work for this exercise will be done in this notebook.

In [17]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data
Similar to previous exercises, we will load CIFAR-10 data from disk.

In [18]:
from cs231n.features import color_histogram_hsv, hog_feature

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # Subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

## Extract Features
For each image we will compute a Histogram of Oriented
Gradients (HOG) as well as a color histogram using the hue channel in HSV
color space. We form our final feature vector for each image by concatenating
the HOG and color histogram feature vectors.

Roughly speaking, HOG should capture the texture of the image while ignoring
color information, and the color histogram represents the color of the input
image while ignoring texture. As a result, we expect that using both together
ought to work better than using either alone. Verifying this assumption would
be a good thing to try for your own interest.

The `hog_feature` and `color_histogram_hsv` functions both operate on a single
image and return a feature vector for that image. The extract_features
function takes a set of images and a list of feature functions and evaluates
each feature function on each image, storing the results in a matrix where
each column is the concatenation of all feature vectors for a single image.

In [19]:
from cs231n.features import *

num_color_bins = 10 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: Add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

Done extracting features for 1000 / 49000 images
Done extracting features for 2000 / 49000 images
Done extracting features for 3000 / 49000 images
Done extracting features for 4000 / 49000 images
Done extracting features for 5000 / 49000 images
Done extracting features for 6000 / 49000 images
Done extracting features for 7000 / 49000 images
Done extracting features for 8000 / 49000 images
Done extracting features for 9000 / 49000 images
Done extracting features for 10000 / 49000 images
Done extracting features for 11000 / 49000 images
Done extracting features for 12000 / 49000 images
Done extracting features for 13000 / 49000 images
Done extracting features for 14000 / 49000 images
Done extracting features for 15000 / 49000 images
Done extracting features for 16000 / 49000 images
Done extracting features for 17000 / 49000 images
Done extracting features for 18000 / 49000 images
Done extracting features for 19000 / 49000 images
Done extracting features for 20000 / 49000 images
Done extr

## Train SVM on features
Using the multiclass SVM code developed earlier in the assignment, train SVMs on top of the features extracted above; this should achieve better results than training SVMs directly on top of raw pixels.

In [20]:
# Use the validation set to tune the learning rate and regularization strength

from cs231n.classifiers.linear_classifier import LinearSVM

learning_rates = [1e-9, 1e-8, 1e-7]
regularization_strengths = [5e4, 5e5, 5e6]

results = {}
best_val = -1
best_svm = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained classifer in best_svm. You might also want to play          #
# with different numbers of bins in the color histogram. If you are careful    #
# you should be able to get accuracy of near 0.44 on the validation set.       #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

for lr in learning_rates:
  for reg in regularization_strengths:
    svm=LinearSVM()
    svm.train(X_train_feats, y_train, learning_rate=lr, reg=reg,
                          num_iters=1500, verbose=True)
    train_acc=np.mean(svm.predict(X_train_feats)==y_train)
    val_acc=np.mean(svm.predict(X_val_feats)==y_val)
    results[(lr,reg)]=[train_acc,val_acc]
    if val_acc>best_val:
      best_val=val_acc
      best_svm=svm
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved: %f' % best_val)

iteration 0 / 1500: loss 82.482080
iteration 100 / 1500: loss 81.024094
iteration 200 / 1500: loss 79.597635
iteration 300 / 1500: loss 78.211247
iteration 400 / 1500: loss 76.833365
iteration 500 / 1500: loss 75.495438
iteration 600 / 1500: loss 74.163757
iteration 700 / 1500: loss 72.886524
iteration 800 / 1500: loss 71.622906
iteration 900 / 1500: loss 70.363597
iteration 1000 / 1500: loss 69.167920
iteration 1100 / 1500: loss 67.966190
iteration 1200 / 1500: loss 66.791254
iteration 1300 / 1500: loss 65.651517
iteration 1400 / 1500: loss 64.531813
iteration 0 / 1500: loss 770.804242
iteration 100 / 1500: loss 632.626619
iteration 200 / 1500: loss 519.542755
iteration 300 / 1500: loss 426.961547
iteration 400 / 1500: loss 351.150874
iteration 500 / 1500: loss 289.112407
iteration 600 / 1500: loss 238.312743
iteration 700 / 1500: loss 196.723253
iteration 800 / 1500: loss 162.675482
iteration 900 / 1500: loss 134.814937
iteration 1000 / 1500: loss 111.994632
iteration 1100 / 1500: lo

In [21]:
# Evaluate your trained SVM on the test set: you should be able to get at least 0.40
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

0.416


(array([  7,  22,  31,  34,  63,  67,  70,  71,  72, 106, 114, 120, 121,
       124, 126, 128, 129, 142, 151, 159, 160, 165, 167, 173, 184, 187,
       201, 202, 211, 218, 246, 262, 265, 295, 306, 310, 312, 326, 342,
       344, 357, 364, 385, 391, 404, 408, 410, 420, 446, 449, 450, 454,
       459, 479, 482, 485, 492, 495, 498, 507, 519, 529, 538, 553, 566,
       577, 586, 603, 629, 630, 635, 674, 682, 688, 694, 695, 697, 701,
       706, 735, 737, 746, 750, 751, 755, 757, 766, 769, 786, 792, 826,
       837, 839, 852, 873, 878, 908, 938, 939, 965, 974, 976, 977, 980,
       985]),)
(array([  1,  15,  40,  45,  57,  62,  69,  76, 101, 108, 111, 150, 162,
       174, 191, 199, 233, 259, 293, 328, 335, 339, 343, 349, 350, 358,
       383, 394, 401, 417, 451, 469, 528, 534, 539, 582, 585, 587, 589,
       595, 606, 611, 621, 666, 675, 680, 683, 700, 725, 734, 741, 770,
       785, 788, 814, 853, 866, 881, 918, 930, 969]),)
(array([ 21,  48,  49,  85, 100, 176, 186, 197, 227, 236, 254, 2

In [22]:
# An important way to gain intuition about how an algorithm works is to
# visualize the mistakes that it makes. In this visualization, we show examples
# of images that are misclassified by our current system. The first column
# shows images that our system labeled as "plane" but whose true label is
# something other than "plane".

examples_per_class = 8
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for cls, cls_name in enumerate(classes):
    print(np.where((y_test != cls) & (y_test_pred == cls)))
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    idxs = np.random.choice(idxs, examples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls_name)
plt.show()

### Inline question 1:
Describe the misclassification results that you see. Do they make sense?


$\color{blue}{\textit Your Answer:}$




## Neural Network on image features
Earlier in this assigment we saw that training a two-layer neural network on raw pixels achieved better classification performance than linear classifiers on raw pixels. In this notebook we have seen that linear classifiers on image features outperform linear classifiers on raw pixels. 

For completeness, we should also try training a neural network on image features. This approach should outperform all previous approaches: you should easily be able to achieve over 55% classification accuracy on the test set; our best model achieves about 60% classification accuracy.

In [23]:
# Preprocessing: Remove the bias dimension
# Make sure to run this cell only ONCE
print(X_train_feats.shape)
X_train_feats = X_train_feats[:, :-1]
X_val_feats = X_val_feats[:, :-1]
X_test_feats = X_test_feats[:, :-1]

print(X_train_feats.shape)

(49000, 155)
(49000, 154)


In [46]:
from cs231n.classifiers.fc_net import TwoLayerNet
from cs231n.solver import Solver

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

data = {
    'X_train': X_train_feats, 
    'y_train': y_train, 
    'X_val': X_val_feats, 
    'y_val': y_val, 
    'X_test': X_test_feats, 
    'y_test': y_test, 
}

net = TwoLayerNet(input_dim, hidden_dim, num_classes)
best_net = None

################################################################################
# TODO: Train a two-layer neural network on image features. You may want to    #
# cross-validate various parameters as in previous sections. Store your best   #
# model in the best_net variable.                                              #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
learning_rates = [1e-1]
num_epochses=[20]
for lr in learning_rates:
    for num_epochs in num_epochses:
        net=TwoLayerNet(input_dim, hidden_dim, num_classes)
        solver = Solver(net, data,
                            update_rule='sgd',
                            optim_config={
                              'learning_rate': lr,
                            },
                            lr_decay=0.95,
                            num_epochs=num_epochs, batch_size=200,
                            print_every=100)
        solver.train()
best_net=TwoLayerNet(input_dim, hidden_dim, num_classes)
best_net.params=solver.best_params


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


(Iteration 1 / 4900) loss: 2.302596
(Epoch 0 / 20) train acc: 0.116000; val_acc: 0.114000
(Iteration 101 / 4900) loss: 2.253618
(Iteration 201 / 4900) loss: 1.794896
(Epoch 1 / 20) train acc: 0.432000; val_acc: 0.387000
(Iteration 301 / 4900) loss: 1.624849
(Iteration 401 / 4900) loss: 1.446331
(Epoch 2 / 20) train acc: 0.497000; val_acc: 0.503000
(Iteration 501 / 4900) loss: 1.581371
(Iteration 601 / 4900) loss: 1.321955
(Iteration 701 / 4900) loss: 1.296979
(Epoch 3 / 20) train acc: 0.528000; val_acc: 0.511000
(Iteration 801 / 4900) loss: 1.347851
(Iteration 901 / 4900) loss: 1.441473
(Epoch 4 / 20) train acc: 0.545000; val_acc: 0.526000
(Iteration 1001 / 4900) loss: 1.252415
(Iteration 1101 / 4900) loss: 1.354666
(Iteration 1201 / 4900) loss: 1.282877
(Epoch 5 / 20) train acc: 0.525000; val_acc: 0.522000
(Iteration 1301 / 4900) loss: 1.207973
(Iteration 1401 / 4900) loss: 1.184932
(Epoch 6 / 20) train acc: 0.527000; val_acc: 0.532000
(Iteration 1501 / 4900) loss: 1.303359
(Iteration

In [47]:
# Run your best neural net classifier on the test set. You should be able
# to get more than 55% accuracy.

y_test_pred = np.argmax(best_net.loss(data['X_test']), axis=1)
test_acc = (y_test_pred == data['y_test']).mean()
print(test_acc)

0.578
